# ***Ednei Cunha Vicente***

Objetivo :
    Esse projeto tem como finalidade a criação de um App Web, com dados do IBGE sobre mortes violentas nas regiões brasileiras.

Fase I : 
    Preprocessamento dos Dados, será necessário para que os dados venham estar apropriados para criação de mapas e gráficos.

Este projeto tem como base registro obtidos pelo IBGE, sobre crimes violentos.

In [ ]:
#importacao das bibliotecas
import folium
import pandas as pd 
import numpy as np
import requests
import re

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Ednei Cunha Vicente" --iversions

In [ ]:
#importacao dataset
df = pd.read_excel('dados/dados_mortes.xlsx')

In [ ]:
#visualização dos dados
df.head()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
#alterar nome das colunas para facilitar nosso trabalho
names = ['Regiao','Homicidio_Doloso_2019',"homicidio_Doloso_2020",'Latrocinio_2019','Latrocinio_2020',
        'Lesao_corporal_2019','Lesao_corporal_2020','Policia_CVLI_2019',"Policia_CVLI_2020",'MDIP_2019',"MDIP_2020",
        "MVI_2019","MVI_2020","Taxa_2019","Taxa_2020",'variacao_2019','variacao_2020']

In [ ]:
#deletar uma coluna que não sera interessante 
df = df.drop([0])

In [ ]:
df.head()

In [ ]:
df.columns = names
df.head()

In [ ]:
df['grupos'].value_counts()

In [ ]:
#deletar coluna que tambem nao sera interessante nessa analise
to_drop = ['grupos','variacao_2020']
df.drop(to_drop,inplace=True,axis=1)

In [ ]:
df.head(10)

In [ ]:
df = df.drop([1,2,3,4,6,30,21])
df.head()

In [ ]:
df.info()

In [ ]:
#usando o camando replace para deixar as variaveis de acordo.
df ['Regiao'] = pd.Categorical(df['Regiao'])
df["Regiao"] = df['Regiao'].replace(to_replace=['Alagoas (5)'],value='Alagoas')
df["Regiao"] = df['Regiao'].replace(to_replace=['Mato Grosso (6)'],value='Mato Grosso')
df["Regiao"] = df['Regiao'].replace(to_replace=['Paraíba (5)'],value='Paraíba')
df["Regiao"] = df['Regiao'].replace(to_replace=['Pernambuco (5)'],value='Pernambuco')
df["Regiao"] = df['Regiao'].replace(to_replace=['Piauí (5)'],value='Piauí')
df["Regiao"] = df['Regiao'].replace(to_replace=['Minas Gerais (5) (7)'],value='Minas Gerais')
df["Regiao"] = df['Regiao'].replace(to_replace=['São Paulo (8)'],value='São Paulo')
df["Regiao"] = df['Regiao'].replace(to_replace=['Rio Grande do Norte (5)'],value='Rio Grande do Norte')
df["Regiao"] = df['Regiao'].replace(to_replace=['Mato Grosso do Sul (5)'],value='Mato Grosso do Sul')
df["Regiao"] = df['Regiao'].replace(to_replace=['Acre (5)'],value='Acre')
df

In [ ]:
headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'User-Agent': 'google-colab',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
}

In [ ]:
# url para solicitação de malhas de todas as Unidades da Federação do Brasil no formato geojson
meshes_url = 'https://servicodados.ibge.gov.br/api/v2/malhas/?resolucao=2&formato=application/vnd.geo+json'

# solicitando dados geojson e atribuindo a variável 
meshes_data = requests.get(meshes_url, headers=headers).json()

In [ ]:
states_url = 'https://servicodados.ibge.gov.br/api/v1/localidades/estados'

In [ ]:
states_data = requests.get(states_url, headers=headers).json()

In [ ]:
# criação de listas para serem preenchidas pelos dados solicitados pelo IBGE
meshes_ids = []
states_ids = []
states_names = []
states_codes = []

# preenchendo informações sobre malhas
for feature in meshes_data['features']:
    meshes_ids.append( str(feature['properties']['codarea']) )

meshes_ids.sort()

# preenchimento de informações sobre Unidades da Federação
for state in states_data:
    states_ids.append( str(state['id']) )
    states_names.append( state['nome'] )
    states_codes.append( state['sigla'] )

states_ids.sort()

# verificando se os números de identificação correspondem
meshes_ids == states_ids

In [ ]:
# criando um dataframe de Unidades Federativas a serem mescladas
states = pd.DataFrame( {'id': states_ids, 'nome': states_names, 'sigla': states_codes} )

# anexando colunas de coordenadas do centroide
states['lat'] = 0
states['lng'] = 0

states.set_index('id', inplace=True)

# recuperando dados do centroide
for feature in meshes_data['features']:    
    centroid = feature['properties']['centroide']
    lat = centroid[1]
    lng = centroid[0]
    
    cod = str(feature['properties']['codarea'])
    
    states.loc[cod,'lat'] = lat
    states.loc[cod,'lng'] = lng

states.reset_index(inplace=True)

In [ ]:
states

In [ ]:
df.info()

In [ ]:
# mesclando dataframes
df = df.merge(states, left_on='Regiao', right_on='nome').drop(columns=['Regiao'])

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df['nome']

In [ ]:
#salvando em csv
df.to_csv("dados/novo_df.csv",index=False)

# ***FIM***